# 💬 Building Software on top of LLMs with Gemini

Welcome! This notebook will guide you through using LLMs via API.

**Before you start**, make sure you have:
1. Your Gemini API key from [aistudio.google.com](https://aistudio.google.com)
2. Added it to Colab Secrets (🔑 icon in left sidebar) with the name `GEMINI_API_KEY`

**Note:** The free tier API allowance from Gemini is quite restrictive. If you find that you have hit your limit, the open source `Gemma` models are all available via the API with higher rate limits.

---
## 🔧 Part 1: Setup

Run this cell first to install the library and configure your API key.

In [ ]:
# Install the Google AI library (only needs to run once)
!pip install -q google-genai

# Import and configure
import google.generativeai as genai
from google.colab import userdata
import json
import time

# Get your API key from Colab secrets
try:
    genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
    print("✅ API key configured successfully!")
except Exception as e:
    print("Error: Could not find API key.")
    print("Make sure you've added GEMINI_API_KEY to your Colab secrets (🔑 icon)")
    print(f"Error details: {e}")

In [ ]:
# Quick test - run this to make sure everything works
model = genai.GenerativeModel('gemini-3-flash-preview')
response = model.generate_content("Say hello in three different languages")
print("Test response:")
print(response.text)
print("\nIf you see greetings above, you're all set!")

---
## Part 2: Try out Gemini Models in the Playground

Before calling models from the API, try out prompting different Gemini models in the AI studio playground here:

https://aistudio.google.com/prompts/new_chat

Some things to experiment with:

- Adjusting levels of thinking
- Adding System Instructions
- Adjusting the temperature
- Enabling "Grounding with Google Search"
- Enabling code execution

---
## 📚 Part 3: Your First API Calls

Now let's explore what you can do with the API!

First let's get a list of all of the different models that are available to use:

In [ ]:
for model in genai.list_models():
  print(model.name)

### Exercise 3.1: Basic API Call

The simplest way to use the API - send a prompt, get a response.

In [ ]:
model = genai.GenerativeModel('gemini-3-flash-preview')

prompt = "Explain what an API is in exactly two sentences."
response = model.generate_content(prompt)

print(response.text)

In [ ]:
# YOUR TURN: Modify the prompt above!
# Try:
#   - Change "two sentences" to "one sentence" or "three bullet points"
#   - Change the topic from APIs to something else

prompt = "YOUR PROMPT HERE"  # <-- Edit this!
response = model.generate_content(prompt)
print(response.text)

### Exercise 3.2: Processing Multiple Items

One of the API's superpowers: processing many items automatically!

In [ ]:
# A list of tech buzzwords we want explained simply
buzzwords = [
    "llm",
    "vibe coding",
]

model = genai.GenerativeModel('gemini-2.5-flash-lite')

print("Tech Jargon Translator")
print("=" * 50)
print()

for word in buzzwords:
    prompt = f"Explain '{word}' in one simple sentence that my grandmother would understand."
    response = model.generate_content(prompt)
    print(f"📌 {word.upper()}")
    print(f"   {response.text}")
    print()

### Exercise 3.3: System Instructions

Give the model a persistent "personality" or set of rules.

In [ ]:
model = genai.GenerativeModel(
    'gemini-3-flash-preview',
    system_instruction="""You are a dramatic movie trailer narrator.

    Rules:
    - Everything you say should sound like it's from an epic movie trailer
    - Use dramatic pauses (written as "...")
    - Keep responses short but INTENSE
    - End with a tagline
    """
)

# Now all responses will follow these rules
response = model.generate_content("What is photosynthesis?")
print(response.text)

In [ ]:
# YOUR TURN: Create your own system prompts and see what you can do!

my_model = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction="""YOUR SYSTEM PROMPT HERE

    Rules:
    - Rule 1
    - Rule 2
    - Rule 3
    """
)

response = my_model.generate_content("What is the weather like today?")
print(response.text)

### Exercise 3.4: Structured Output with JSON

Get data in a specific format you can use in your code.

In [ ]:
import json

food = "cookies"

prompt = f"""Give me a recipe for {food}

Return a JSON object with these exact fields:
{{
    "recipe_name": "...",
    "ingredients": ["...", "...", "..."],
    "instructions": ["...", "...", "..."],
}}

Return ONLY the JSON, no other text.
"""

model = genai.GenerativeModel(
    'gemini-2.5-flash',
)

response = model.generate_content(prompt, generation_config=genai.GenerationConfig(
    response_mime_type="application/json",
))

recipe = json.loads(response.text)

print("Name:", recipe["recipe_name"])

print("Ingredients:")
for ingredient in recipe["ingredients"]:
    print(f"  - {ingredient}")

print("Instructions:")
for step in recipe["instructions"]:
    print(f"  {step}")

In [ ]:
# YOUR TURN: Create a JSON schema to return some structured data for an LLM task

prompt = f"""INSERT PROMPT HERE

Return a JSON object with these exact fields:
{{
    "technology_name": "...",
    "maturity_level": "emerging/growing/mature",
    "time_to_mainstream": "...",
    "top_3_benefits": ["...", "...", "..."],
    "top_3_challenges": ["...", "...", "..."],
    "confidence_score": 0.0 to 1.0
}}

Return ONLY the JSON, no other text.
"""

model = genai.GenerativeModel(
    'gemini-2.5-flash',
)

response = model.generate_content(prompt, generation_config=genai.GenerationConfig(
    response_mime_type="application/json",
))

recipe = json.loads(response.text)
print(json.dumps(recipe, indent=2))

### Exercise 3.6: Images via the API 🖼️

You can also send images to the API! First, upload an image to Colab:
1. Run the cell below and click "Browse" to upload an image from your computer

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import PIL.Image

model = genai.GenerativeModel('gemini-2.5-flash-lite')

filename = list(uploaded.keys())[0]
image = PIL.Image.open(filename)

# Ask about the image
response = model.generate_content([
    "Describe this image in a fun, creative way.",
    image
])

print(response.text)

In [ ]:
# Try different prompts with the same image!

prompt = "Write a haiku inspired by this image."

response = model.generate_content([prompt, image])
print(f"📸 {prompt}")
print(f"   {response.text}\n")

In [ ]:
# Exercise: Try out gemma-3-27b-it or some of the other available models to see how they differ in results

### Exercise 3.7: Tool Use 🔨

Tool calling (also known as function calling) allows LLMs to interact with external systems by calling functions you define. Instead of just generating text, the model can:

1. Recognise when a user's request requires external data or actions
2. Generate a structured function call with the appropriate arguments
3. Use the function's response to formulate its final answer

Run the cell below to see how the `get_weather` tool is called. Swap out the prompts and see if the tool is still called for questions that are not about the weather.

In [ ]:
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    # Fake weather data for demo purposes
    print(f"TOOL: Calling the weather tool for {city}")
    weather_data = {
        "london": "Cloudy, 12°C",
        "paris": "Sunny, 18°C",
        "tokyo": "Rainy, 15°C",
    }
    return weather_data.get(city.lower(), f"Weather data not available for {city}")


# Create the model with the tool
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    tools=[get_weather],
)

chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message("What's the weather like in Paris?")
print(response.text)

---
## 🛠️ Part 4: Build Something Useful!

Build a tool to parse line items from receipts. For each line item on the receipt, parse the description and the amount.

**Use the sample code above along with the Gemini assistant built into Colab or any other LLMs you wish to help you out!**

When you're done, find other images of receipts on the web to test the parser out and see how well it performs.

In [ ]:
from io import BytesIO
import requests
from PIL import Image
from IPython.display import display

url = "https://www.thesun.ie/wp-content/uploads/sites/3/2023/09/dunens1.png"
response = requests.get(url)
img = Image.open(BytesIO(response.content))
display(img)